## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.79,-152.41,42.80,87,90,16.11,light rain
1,1,East London,ZA,-33.02,27.91,70.27,91,100,14.70,light rain
2,2,Punta Arenas,CL,-53.15,-70.92,62.60,55,75,27.51,broken clouds
3,3,Tasiilaq,GL,65.61,-37.64,26.60,68,100,8.05,light snow
4,4,Bluff,NZ,-46.60,168.33,56.75,76,49,14.34,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
674,674,Poronaysk,RU,49.22,143.12,17.40,89,100,2.84,overcast clouds
675,675,Rochegda,RU,62.80,43.25,33.42,93,100,6.49,overcast clouds
676,676,Bambanglipuro,ID,-7.95,110.28,78.64,82,100,2.28,light rain
677,677,Shenjiamen,CN,29.96,122.30,57.15,77,98,14.52,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Galle,LK,6.04,80.22,77.00,84,76,3.18,broken clouds
12,12,Avarua,CK,-21.21,-159.78,84.20,74,93,17.22,overcast clouds
14,14,Santa Maria Da Vitoria,BR,-13.39,-44.19,75.97,67,34,2.10,scattered clouds
15,15,Jaciara,BR,-15.97,-54.97,80.64,64,94,2.68,overcast clouds
18,18,Atuona,PF,-9.80,-139.03,78.44,74,1,15.61,clear sky
...,...,...,...,...,...,...,...,...,...,...
663,663,Kismayo,SO,-0.36,42.55,80.29,82,0,10.02,clear sky
667,667,Monrovia,LR,6.30,-10.80,77.00,94,40,7.29,scattered clouds
668,668,Governador Valadares,BR,-18.85,-41.95,75.15,84,100,4.92,overcast clouds
672,672,Carutapera,BR,-1.20,-46.02,79.81,79,97,8.55,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
# Populate None as the value under "Hotel Name" column initially
hotel_df["Hotel Name"] = None
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Galle,LK,77.00,broken clouds,6.04,80.22,None
12,Avarua,CK,84.20,overcast clouds,-21.21,-159.78,None
14,Santa Maria Da Vitoria,BR,75.97,scattered clouds,-13.39,-44.19,None
15,Jaciara,BR,80.64,overcast clouds,-15.97,-54.97,None
18,Atuona,PF,78.44,clear sky,-9.80,-139.03,None
...,...,...,...,...,...,...,...
663,Kismayo,SO,80.29,clear sky,-0.36,42.55,None
667,Monrovia,LR,77.00,scattered clouds,6.30,-10.80,None
668,Governador Valadares,BR,75.15,overcast clouds,-18.85,-41.95,None
672,Carutapera,BR,79.81,overcast clouds,-1.20,-46.02,None


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")
        

Hotel not found... skipping.53
Hotel not found... skipping.101
Hotel not found... skipping.146
Hotel not found... skipping.147
Hotel not found... skipping.280
Hotel not found... skipping.367
Hotel not found... skipping.420
Hotel not found... skipping.512
Hotel not found... skipping.538
Hotel not found... skipping.593
Hotel not found... skipping.635


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Galle,LK,77.00,broken clouds,6.04,80.22,Amangalla
12,Avarua,CK,84.20,overcast clouds,-21.21,-159.78,Paradise Inn
14,Santa Maria Da Vitoria,BR,75.97,scattered clouds,-13.39,-44.19,Pousada do Rio Corrente
15,Jaciara,BR,80.64,overcast clouds,-15.97,-54.97,Taba Hotel
18,Atuona,PF,78.44,clear sky,-9.80,-139.03,Villa Enata
...,...,...,...,...,...,...,...
663,Kismayo,SO,80.29,clear sky,-0.36,42.55,Agan-Hotel
667,Monrovia,LR,77.00,scattered clouds,6.30,-10.80,Eco-Hotel Hall & Restaurant
668,Governador Valadares,BR,75.15,overcast clouds,-18.85,-41.95,Hotel Panorama
672,Carutapera,BR,79.81,overcast clouds,-1.20,-46.02,Lidera


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# ********* Create a Customer Travel Destinations Map complete ***********